# Integrated Flask and Database Assignment #
## Paul Chao ##

### LevelDB locks ###
- Race Condition
  ![](http://blogs.windriver.com/wp-content/uploads/images-o/6a00d83451f5c369e20133f0ed95e4970b-pi.png)
- LevelDB [locks](https://github.com/google/leveldb) for each process
  ```
  Limitations: 
  
  Only a single process (possibly multi-threaded) can access a particular database at a time.
  ```
- Lock file mechanism

#### 一件事做完才做下一件事 !! 

### Flask Debugger forks ###
- it activates the debugger
- it activates the automatic reloader
- it enables the debug mode on the Flask application.

[flask](https://github.com/pallets/flask/blob/c74f46979a8b8358437bd7f76e478d04248a9c72/flask/app.py#L892)-->[werkzeug](https://github.com/pallets/werkzeug/blob/master/werkzeug/serving.py#L734)-->[werkzeug/debug](https://github.com/pallets/werkzeug/blob/0bc61df6e1ae9f2ffdf5d066aa3cd9d5ebcb307d/werkzeug/debug/__init__.py#L198)

[ref](http://werkzeug.pocoo.org/docs/0.13/serving/)

### DEBUG MODE is OFF ###
``` Python
from flask import Flask
from db_util import init, insert, delete, update, search, dump


app = Flask(__name__)

@app.route('/')
def aloha():
    return ("Aloha! Paul!")

@app.route('/test')
def test():
    return ("test! Paul!")

@app.route('/test2/<string:var>')
def test2(var):
    test = str(var) + " is the path"
    return "test2 {}".format(test)

@app.route('/db/create/<string:var>')
def db_list(var):
    if db is not None:
        insert(db, str(var), 'test')
    return ("Create item {} successfully.".format(var))

@app.route('/db/dump')
def db_dump():
    if db is not None:
        item_list = dump(db)
    return ("dump item\n {}".format(item_list))

if __name__ == '__main__':
    db = init('mydb')
    app.run()
```

### DEBUG MODE is ON

``` Python
from flask import Flask
from db_util import init, insert, delete, update, search, dump


app = Flask(__name__)
global db

@app.route('/')
def aloha():
    return ("Aloha! Paul!")

@app.route('/test')
def test():
    return ("test! Paul!")

@app.route('/test2/<string:var>')
def test2(var):
    test = str(var) + " is the path"
    return "test2 {}".format(test)

@app.route('/db/create/<string:var>')
def db_list(var):
    global db
    if db is None:
        db = init('mydb')

    if db is not None:
        #db = init('mydb')
        insert(db, str(var), 'test')
    return ("Create item {} successfully.".format(var))

@app.route('/db/dump')
    global db
    if db is None:
        db = init('mydb')

    if db is not None:
        item_list = dump(db)
    return ("dump item\n {}".format(item_list))

if __name__ == '__main__':
    #db = init('mydb')
    db = None
    app.run(debug=True)
```

### db_util modify dump ###
``` Python
import leveldb

# converting functions
def cvt_to_bytes(string):
    # leave type check for system exceptions
    return string.encode('ascii')


def cvt_to_string(bytestring):
    # leave type check for system exceptions
    return bytestring.decode('utf-8')


def cvt_b(key, value):
    key_b = key
    value_b = value
    if isinstance(key, str):
        key_b = cvt_to_bytes(key)
    if isinstance(value, str):
        value_b = cvt_to_bytes(value)
    return key_b, value_b


def cvt_s(key, value):
    key_s = key
    value_s = value

    if isinstance(key, (bytes, bytearray)):
        key_s = cvt_to_string(key)
    if isinstance(value, (bytes, bytearray)):
        value_s = cvt_to_string(value)
    return key_s, value_s


# database functions
def init(filename):
    db = leveldb.LevelDB(filename)
    return db


def insert(db, key, value):
    key_b, name_b = cvt_b(key, value)
    db.Put(key_b, name_b)


def delete(db, key):
    key_b = cvt_to_bytes(key)
    db.Delete(key_b)


def update(db, key, value):
    key_b, name_b = cvt_b(key, value)
    db.Put(key_b, name_b)


def search(db, key):
    key_b = cvt_to_bytes(key)
    value = db.Get(key_b)
    value_str = cvt_to_string(value)
    return value_str


def dump(db):
    # add container
    item_list = []
    for key, value in db.RangeIter():
        key_s, value_s = cvt_s(key, value)
        item_list.append((key_s, value))
        print (key_s, value_s)
    return item_list
```